<a href="https://colab.research.google.com/github/lindanimoyo/2018-May-Microbial-Community-Analysis-Workshop_UCB/blob/master/Plants_R_REST_course.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ensembl REST API course

Welcome to the Ensembl REST API course. This Jupyter notebook contains all you need to complete this course, including sample code, space to write your own code and sample answers, which can all be run within the notebook in Python.

To run the code within a cell, select it and hit Run.

## REST calls in the browser

Our first REST call will be the [ping call](http://rest.ensembl.org/documentation/info/ping) which checks your connection to the database:

[http://rest.ensembl.org/info/ping?content-type=application/json](http://rest.ensembl.org/info/ping?content-type=application/json)

Our next call will [get information about a gene](http://rest.ensembl.org/documentation/info/lookup), using the Ensembl gene ID:

[http://rest.ensembl.org/lookup/id/TraesCS3D02G007500?content-type=application/json](http://rest.ensembl.org/lookup/id/TraesCS3D02G007500?content-type=application/json)

## Exercises

The first set of exercises just use URLs in the browser. You cannot do these within the Python notebook, however we have provided a box where you can note them down.

1. Find an endpoint which you can use to lookup information about a gene using its symbol.
2. Create a URL to find information about the gene BCH1 in barley (Hordeum vulgare).
3. Expand your results to include information about transcripts.

Answers:

1.

2.

3.

# Making requests with R

To make a request, you'll need to specify the server and extension. Printing the request will give you the response code.

In [ ]:
library(httr)
library(jsonlite)

server <- "http://rest.ensembl.org"
ext <- "/lookup/id/TraesCS3D02G007500"

r <- GET(paste(server, ext, sep = ""), accept("application/json"))

r

In most cases, you only want to see the response code of failed requests, so you can use the httr module stop_for_status

In [ ]:
library(httr)
library(jsonlite)

server <- "http://rest.ensembl.org"
ext <- "/lookup/id/TraesCS3D02G007500"

r <- GET(paste(server, ext, sep = ""), accept("application/json"))

stop_for_status(r)



The httr library includes a module that gets the text of the response, and the jsonlite library can use this as input to decode the json. I've used the prettify command to print, which makes my json easy to read. You'll find this useful during the exercises to see how the json looks.

In [ ]:
library(httr)
library(jsonlite)

server <- "http://rest.ensembl.org"
ext <- "/lookup/id/TraesCS3D02G007500"

r <- GET(paste(server, ext, sep = ""), accept("application/json"))

stop_for_status(r)

prettify(content(r, "text", encoding = "UTF-8"))

The helper function allows you to call the request, check the status and decode the json in a single line in your script. If you're using lots of REST calls in your script, creating the function at the beginning of your script will save you a lot of time

In [ ]:
library(httr)
library(jsonlite)

fetch_endpoint <- function(server, request, content_type){

    r <- GET(paste(server, request, sep = ""), accept(content_type))

    stop_for_status(r)

    if (content_type == 'application/json'){
        return (fromJSON(content(r, "text", encoding = "UTF-8")))
    } else {
        return (content(r, "text", encoding = "UTF-8"))
    }
}

server <- "http://rest.ensembl.org/"
ext <- "/lookup/id/TraesCS3D02G007500"
con <- "application/json"
get_gene <- fetch_endpoint(server, ext, con)

prettify(toJSON(get_gene))

## Exercises 2

1. Write a script to **lookup** the gene called *BCH1* in barley and print the results in json.

In [ ]:
# Exercise 2

## Exercises 2 – answers

1. Write a script to **lookup** the gene called *BCH1* in barley and print the results in json.

In [ ]:
library(httr)
library(jsonlite)

fetch_endpoint <- function(server, request, content_type){

    r <- GET(paste(server, request, sep = ""), accept(content_type))

    stop_for_status(r)

    if (content_type == 'application/json'){
        return (fromJSON(content(r, "text", encoding = "UTF-8")))
    } else {
        return (content(r, "text", encoding = "UTF-8"))
    }
}

# define the gene name
gene_name <- "BCH1"

# define the general URL parameters
server <- "http://rest.ensembl.org/"
con <- "application/json"

# define REST query to get the gene ID from the gene name
ext_get_lookup <- paste("lookup/symbol/hordeum_vulgare/", gene_name, "?", sep ="")

get_lookup <- fetch_endpoint(server, ext_get_lookup, con)

prettify(toJSON(get_lookup))

# Using results

The fromJSON module we used to pull out the data converts it to a dataframe, so you can pull out a single datapoint using the key.

```
{'assembly_name': 'IWGSC',
 'biotype': 'protein_coding',
 'db_type': 'core',
 'end': 2603119,
 'id': 'TraesCS3D02G007500',
 'logic_name': 'iwgsc_high_conf',
 'object_type': 'Gene',
 'seq_region_name': '3D',
 'source': 'IWGSC',
 'species': 'triticum_aestivum',
 'start': 2600356,
 'strand': -1}
```


We can add this to our previous script:

In [ ]:
library(httr)
library(jsonlite)

fetch_endpoint <- function(server, request, content_type){

    r <- GET(paste(server, request, sep = ""), accept(content_type))

    stop_for_status(r)

    if (content_type == 'application/json'){
        return (fromJSON(content(r, "text", encoding = "UTF-8")))
    } else {
        return (content(r, "text", encoding = "UTF-8"))
    }
}

server <- "http://rest.ensembl.org/"
ext <- "lookup/id/TraesCS3D02G007500?"
con <- "application/json"
get_gene <- fetch_endpoint(server, ext, con)

logic_name <- get_gene$logic_name

cat(logic_name)

## Exercises 3

1\. Write a script to lookup the gene called BCH1 in barley and print the stable ID of this gene.

In [ ]:
# Exercise 3.1

2\. Get all variants overlapping the *Arabidopsis thaliana* gene AT3G52430. For each variant print:

   a. The variant ID.
   
   b. The source.
   
   c. The alleles.

In [ ]:
# Exercise 3.2

3\. Get the maize (*Zea mays*) homologue of the *Arabidopsis thaliana* CP33 and print the ID and sequence of both.

Note that the JSON for the endpoint you need is several layers deep, containing nested lists (appear as square brackets [ ] in the JSON) and key value sets (appear as curly brackets { } in the JSON). Prettify comes in very useful here for the intermediate stage when you're trying to work out the json.

In [ ]:
# Exercise 3.3

## Exercises 3 – answers

1\. Write a script to lookup the gene called BCH1 in barley and print the stable ID of this gene.

In [ ]:
library(httr)
library(jsonlite)

fetch_endpoint <- function(server, request, content_type){

    r <- GET(paste(server, request, sep = ""), accept(content_type))

    stop_for_status(r)

    if (content_type == 'application/json'){
        return (fromJSON(content(r, "text", encoding = "UTF-8")))
    } else {
        return (content(r, "text", encoding = "UTF-8"))
    }
}

# define the gene name
gene_name <- "BCH1"

# define the general URL parameters
server <- "http://rest.ensembl.org/"
con <- "application/json"

# define REST query to get the gene ID from the gene name
ext_get_lookup <- paste("lookup/symbol/hordeum_vulgare/", gene_name, "?", sep ="")

get_lookup <- fetch_endpoint(server, ext_get_lookup, con)

stable_id <- get_lookup$id

cat(stable_id)

2\. Get all variants overlapping the *Arabidopsis thaliana* gene AT3G52430. For each variant print:

   a. The variant ID.
   
   b. The source.
   
   c. The alleles.

In [ ]:
library(httr)
library(jsonlite)

fetch_endpoint <- function(server, request, content_type){

    r <- GET(paste(server, request, sep = ""), accept(content_type))

    stop_for_status(r)

    if (content_type == 'application/json'){
        return (fromJSON(content(r, "text", encoding = "UTF-8")))
    } else {
        return (content(r, "text", encoding = "UTF-8"))
    }
}

# define the URL parameters
server <- "http://rest.ensembl.org/"
con <- "application/json"
ext_overlap <- "/overlap/id/AT3G52430?feature=variation"

get_overlap <- fetch_endpoint(server, ext_overlap, con)

flat_get_overlap <- flatten(get_overlap, recursive = TRUE)

flat_get_overlap[, c("id", "source", "alleles")]

3\. Get the maize (*Zea mays*) homologue of the *Arabidopsis thaliana* CP33 and print the ID and the aligned sequence of both.

Note that the JSON for the endpoint you need is several layers deep, containing nested lists (appear as square brackets [ ] in the JSON) and key value sets (appear as curly brackets { } in the JSON).

In [ ]:
library(httr)
library(jsonlite)

fetch_endpoint <- function(server, request, content_type){

    r <- GET(paste(server, request, sep = ""), accept(content_type))

    stop_for_status(r)

    if (content_type == 'application/json'){
        return (fromJSON(content(r, "text", encoding = "UTF-8")))
    } else {
        return (content(r, "text", encoding = "UTF-8"))
    }
}

gene <- "CP33"

# define the URL parameters
server <- "http://rest.ensembl.org/"
con <- "application/json"
ext_hom <- paste("homology/symbol/arabidopsis_thaliana/", gene, "?target_species=zea_mays", sep = "")

get_hom <- fetch_endpoint(server, ext_hom, con)

homologies <- get_hom$data$homologies

for (homology in homologies) {
    homology

    source_id <- homology$source$id
    source_seq <- homology$source$align_seq
    source_species <- homology$source$species

    target_id <- homology$target$id
    target_seq <- homology$target$align_seq
    target_species <- homology$target$species
    
    cat(">", source_id, source_species, "\n", source_seq, "\n>", target_id, target_species, "\n", target_seq)
    }



# Other content types

If you specify another content type (not json), the helper function will get you this as text
This can be used to get:
* Sequence in FASTA
* Gene trees and homologues in various formats
* Alignments

In [ ]:
if (content_type == 'application/json'){
    return (fromJSON(content(r, "text", encoding = "UTF-8")))
} else {
    return (content(r, "text", encoding = "UTF-8"))
}

The [REST documentation](https://github.com/Ensembl/ensembl-rest/wiki/Output-formats) lists how you specify the output formats.

For example, to get genome features in BED, you need to specify the content type as `text/x-bed` not just `bed`:

In [ ]:
library(httr)
library(jsonlite)

fetch_endpoint <- function(server, request, content_type){

    r <- GET(paste(server, request, sep = ""), accept(content_type))

    stop_for_status(r)

    if (content_type == 'application/json'){
        return (fromJSON(content(r, "text", encoding = "UTF-8")))
    } else {
        return (content(r, "text", encoding = "UTF-8"))
    }
}

locus <- "2:6129736..6845617"
server <- "http://rest.ensembl.org/"
ext_get_bed <- paste("overlap/region/oryza_sativa/", locus, "?feature=repeat", sep = "")
con <- "text/x-bed"

get_bed <- fetch_endpoint(server, ext_get_bed, con)

# print the bed file
cat(get_bed)

## Exercises 4 

1\. Get the gene tree predicted for the gene Pp3c2_11370 in full nh format. 

In [ ]:
# Exercise 4.1

2\. Get the sequence of the gene TraesCS4A02G446800 in FASTA format.

In [ ]:
# Exercise 4.2

## Exercises 4 – answers 

1\. Get the gene tree predicted for the gene Pp3c2_11370 in full nh format. 

In [ ]:
library(httr)

fetch_endpoint <- function(server, request, content_type){

    r <- GET(paste(server, request, sep = ""), accept(content_type))

    stop_for_status(r)

    if (content_type == 'application/json'){
        return (fromJSON(content(r, "text", encoding = "UTF-8")))
    } else {
        return (content(r, "text", encoding = "UTF-8"))
    }
}

gene_id <- "Pp3c2_11370"

# define the general URL parameters
server <- "http://rest.ensembl.org/"
ext_gt <- paste("genetree/member/id/", gene_id, "?nh_format=full;", sep = "")
gt_content_type <- "text/x-nh"
get_gt <- fetch_endpoint(server, ext_gt, gt_content_type)

cat(get_gt)

2\. Get the sequence of the gene TraesCS4A02G446800 in FASTA format.

In [ ]:
library(httr)
library(jsonlite)

fetch_endpoint <- function(server, request, content_type){

    r <- GET(paste(server, request, sep = ""), accept(content_type))

    stop_for_status(r)

    if (content_type == 'application/json'){
        return (fromJSON(content(r, "text", encoding = "UTF-8")))
    } else {
        return (content(r, "text", encoding = "UTF-8"))
    }
}

gene <- "TraesCS4A02G446800"
server <- "http://rest.ensembl.org/"
ext_get_seq <- paste("/sequence/id/", gene, "?", sep = "")
con <- "text/x-fasta"

get_seq <- fetch_endpoint(server, ext_get_seq, con)

# print the gene name, ID and sequence
cat(get_seq)

# Linking endpoints together

If you can pull a datapoint from the json, you can use it as input for another endpoint.

For example, this script gets the genomic locus of a gene then looks up repeats overlapping it:

In [ ]:
library(httr)
library(jsonlite)

fetch_endpoint <- function(server, request, content_type){

    r <- GET(paste(server, request, sep = ""), accept(content_type))

    stop_for_status(r)

    if (content_type == 'application/json'){
        return (fromJSON(content(r, "text", encoding = "UTF-8")))
    } else {
        return (content(r, "text", encoding = "UTF-8"))
    }
}

server <- "http://rest.ensembl.org/"
gene_ext <- "lookup/id/TraesCS3D02G007500?"
json <- "application/json"
get_gene <- fetch_endpoint(server, gene_ext, json)

locus = paste(get_gene$species, "/", get_gene$seq_region_name, ":",get_gene$start, "..", get_gene$end, sep="")

ext_get_bed <- paste("overlap/region/", locus, "?feature=repeat", sep = "")

get_bed <- fetch_endpoint(server, ext_get_bed, "text/x-bed")

# print the bed file
cat(get_bed)

## Exercises 5

1\. Using the script from 3.1, add a call to fetch and print the sequence for the gene BCH1 in FASTA format.

In [ ]:
# Exercise 5.1

2\. Print the ID of any variants that overlap the region 5000 bp upstream of the Brachypodium BRADI_5g16840v3 gene. (Hints: get the gene ID first, then check the strand of the gene to see which way is upstream.)

In [ ]:
# Exercise 5.2

## Exercises 5 - answers

1\. Using the script from 3.1, add a call to fetch and print the sequence for the gene BCH1 in FASTA format.

In [ ]:
library(httr)
library(jsonlite)

fetch_endpoint <- function(server, request, content_type){

    r <- GET(paste(server, request, sep = ""), accept(content_type))

    stop_for_status(r)

    if (content_type == 'application/json'){
        return (fromJSON(content(r, "text", encoding = "UTF-8")))
    } else {
        return (content(r, "text", encoding = "UTF-8"))
    }
}

# define the gene name
gene_name <- "BCH1"

# define the general URL parameters
server <- "http://rest.ensembl.org/"
con <- "application/json"

# define REST query to get the gene ID from the gene name
ext_get_lookup <- paste("lookup/symbol/hordeum_vulgare/", gene_name, "?", sep ="")

get_lookup <- fetch_endpoint(server, ext_get_lookup, con)

stable_id <- get_lookup$id

# define the REST query to get the sequence from the gene
ext_get_seq <- paste("sequence/id/", get_lookup$id, "?", sep = "")
get_seq <- fetch_endpoint(server, ext_get_seq, 'text/x-fasta')

# print the gene name, ID and sequence
cat(get_seq)

2\. Print the ID of any variants that overlap the region 5000 bp upstream of the Brachypodium BRADI_5g16840v3 gene. (Hints: get the gene ID first, then check the strand of the gene to see which way is upstream.)

In [ ]:
library(httr)
library(jsonlite)

fetch_endpoint <- function(server, request, content_type){

    r <- GET(paste(server, request, sep = ""), accept(content_type))

    stop_for_status(r)

    if (content_type == 'application/json'){
        return (fromJSON(content(r, "text", encoding = "UTF-8")))
    } else {
        return (content(r, "text", encoding = "UTF-8"))
    }
}

server <- "http://rest.ensembl.org/"
con <- "application/json"

gene_name <- "BRADI_5g16840v3"
ext_get_lookup <- paste("lookup/id/", gene_name, "?", sep = "")
get_lookup <- fetch_endpoint(server, ext_get_lookup, con)

if (get_lookup$strand == 1){
    locus <- paste (get_lookup$seq_region_name, ":", get_lookup$start - 5000, "-", get_lookup$start, sep = "")

} else {
    locus <- paste (get_lookup$seq_region_name, ":", get_lookup$end, "-", get_lookup$end + 5000, sep = "")
}

overlap_ext <- paste ("overlap/region/brachypodium_distachyon/", locus, "?feature=variation;", sep = "")

get_overlap <- fetch_endpoint(server, overlap_ext, con)

cat(get_overlap$id)

# Using POST

POST allows you to run a query with multiple inputs at once. The output will be a list of data-frames.

In [ ]:
library(httr)
library(jsonlite)
 
server <- "https://rest.ensembl.org"
ext <- "/lookup/id"
genes <- c("ENSG00000157764", "ENSG00000248378")
body_values <- toJSON(list(ids=genes))

r <- POST(paste(server, ext, sep = ""), content_type("application/json"), accept("application/json"), body = body_values)

prettify(content(r, "text", encoding = "UTF-8"))

There is a helper function in POST. You can specify both helper functions in your script and use whichever one you need.

In [ ]:
fetch_endpoint_POST <- function(server, request, data, content_type='application/json'){

    r <- POST(paste(server, request, sep = ""), content_type("application/json"), accept("application/json"), body = data)

    stop_for_status(r)

    if (content_type == 'application/json'){
        return (fromJSON(content(r, "text", encoding = "UTF-8")))
    } else {
        return (content(r, "text", encoding = "UTF-8"))
    }
}

### Optional parameters

In order to add optional parameters to your POST query, you can just add them onto the extention with a slash. For example if you wanted to mask UTRs when running the [sequence_id_post](http://rest.ensembl.org/documentation/info/sequence_id_post) endpoint, you could specify your extension as:

        `ext <- "sequence/id/mask_feature=1"`


### Input

Your input list for POST queries need to be a JSON list. You can create this from a list in R using the [jsonlite library](https://www.rdocumentation.org/packages/jsonlite/versions/1.5):

        `data <- toJSON(list(ids=mylist))`

### Output

The Output from POST queries will be a list of dataframes.

## Example

The following scripts inputs a list of variants in HGVS format into the VEP and gets out the IDs of known colocated variants, including failed variants (an optional parameter):

In [ ]:
library(httr)
library(jsonlite)

fetch_endpoint <- function(server, request, content_type){

    r <- GET(paste(server, request, sep = ""), accept(content_type))

    stop_for_status(r)

    if (content_type == 'application/json'){
        return (fromJSON(content(r, "text", encoding = "UTF-8")))
    } else {
        return (content(r, "text", encoding = "UTF-8"))
    }
}

fetch_endpoint_POST <- function(server, request, data, content_type='application/json'){

    r <- POST(paste(server, request, sep = ""), content_type("application/json"), accept("application/json"), body = data)

    stop_for_status(r)

    if (content_type == 'application/json'){
        return (fromJSON(content(r, "text", encoding = "UTF-8")))
    } else {
        return (content(r, "text", encoding = "UTF-8"))
    }
}

# define the server, extension and content type
server <- "http://rest.ensembl.org/"
con <- "application/json"
vep_ext <- "vep/triticum_aestivum/hgvs/failed=1"

# create the list of HGVS annotations
hgvs <- c("TraesCS4A02G446800.1:c.92C>T", "TraesCS4A02G446800.1:c.629T>A", "TraesCS4A02G446800.1:c.695A>G")

# convert the list into json format
hgvs_json <- toJSON(list(hgvs_notations=hgvs))

# run a post query with the list of variants
post_vep <- fetch_endpoint_POST(server, vep_ext, hgvs_json, con)

#
for (i in 1:nrow(post_vep)) {
    variant <- post_vep[i, ]
    input <- variant$input
    colocated <- variant$colocated_variants
    for (col in colocated) {
        variant_ids = col$id
    }
    print (paste(input, variant_ids, sep = ": "))
}

## Exercises 6

1\. Fetch the all the transcripts of barley BCH1 using the lookup function. Fetch the cDNA sequences of all transcripts using a single POST request, and print in FASTA format.

In [ ]:
# Exercise 6.1

2\. You have the following list of *Sorghum bicolor* variants:
```tmp_3_5074295_G_A, tmp_3_5074341_T_G, tmp_3_5074428_A_CG, tmp_3_5074453_C_A, S10_10039859```. Get the variant class, source and the most_severe_consequence for all variants using the variation POST endpoint.

In [ ]:
# Exercise 6.2

## Exercises 6 – answers

1\. Fetch the all the transcripts of barley BCH1 using the lookup function. Fetch the cDNA sequences of all transcripts using a single POST request, and print in FASTA format.

In [ ]:
library(httr)
library(jsonlite)

fetch_endpoint <- function(server, request, content_type){

    r <- GET(paste(server, request, sep = ""), accept(content_type))

    stop_for_status(r)

    if (content_type == 'application/json'){
        return (fromJSON(content(r, "text", encoding = "UTF-8")))
    } else {
        return (content(r, "text", encoding = "UTF-8"))
    }
}

fetch_endpoint_POST <- function(server, request, data, content_type='application/json'){

    r <- POST(paste(server, request, sep = ""), content_type("application/json"), accept("application/json"), body = data)

    stop_for_status(r)

    if (content_type == 'application/json'){
        return (fromJSON(content(r, "text", encoding = "UTF-8")))
    } else {
        return (content(r, "text", encoding = "UTF-8"))
    }
}

# define the gene name
gene_name <- "BCH1"
transcripts <- vector()

# define the general URL parameters
server <- "http://rest.ensembl.org/"
con <- "application/json"

# define REST query to get the gene ID from the gene name
ext_get_gene <- paste("lookup/symbol/hordeum_vulgare/", gene_name, "?expand=1;", sep ="")

get_gene <- fetch_endpoint(server, ext_get_gene, con)

# make a vector of the transcripts then convert to a json object
transcripts <- get_gene$Transcript$id
data <- toJSON(list(ids=transcripts))

ext_sequence <- '/sequence/id/'
sequences <- fetch_endpoint_POST(server, ext_sequence, data, con)

sequences[, c("id", "seq")]

2\. You have the following list of *Sorghum bicolor* variants:
```tmp_3_5074295_G_A, tmp_3_5074341_T_G, tmp_3_5074428_A_CG, tmp_3_5074453_C_A, S10_10039859```. Get the variant class, source and the most_severe_consequence for all variants using the variant POST endpoint.

In [ ]:
library(httr)
library(jsonlite)

fetch_endpoint <- function(server, request, content_type){

    r <- GET(paste(server, request, sep = ""), accept(content_type))

    stop_for_status(r)

    if (content_type == 'application/json'){
        return (fromJSON(content(r, "text", encoding = "UTF-8")))
    } else {
        return (content(r, "text", encoding = "UTF-8"))
    }
}

fetch_endpoint_POST <- function(server, request, data, content_type='application/json'){

    r <- POST(paste(server, request, sep = ""), content_type("application/json"), accept("application/json"), body = data)

    stop_for_status(r)

    if (content_type == 'application/json'){
        return (fromJSON(content(r, "text", encoding = "UTF-8")))
    } else {
        return (content(r, "text", encoding = "UTF-8"))
    }
}

# define the general URL parameters
server <- "http://rest.ensembl.org/"
con <- "application/json"

# make a vector listing all the variants then convert to a json list
variants <- c("tmp_3_5074295_G_A", "tmp_3_5074341_T_G", "tmp_3_5074428_A_CG", "tmp_3_5074453_C_A", "S10_10039859")
data <- toJSON(list(ids=variants))

# run a post query with the list of variants
var_ext <- "variation/sorghum_bicolor"
post_variants <- fetch_endpoint_POST(server, var_ext, data, con)

for (variant in post_variants){
    print( paste(variant$name, variant$var_class, variant$source, variant$most_severe_consequence, sep = ", "))
}

# Rate limiting

Requests are rate limited to prevent a single user from monopolising the resources.

Response headers show we are allowed 55000 requests over an hour (3600 seconds): an average 15 requests per second.

## Exercises 7

This script queries the ping endpoint 25 times, printing the count, the HTTP Status Code, and the X-RateLimit-Remaining header each time.

1\. Increase the number of loops, do you start to get 429 errors?

2\. Can you add in a step to make it wait a few seconds every iteration? Or every 100 iterations?

In [ ]:
# Exercise 7.1
library(httr)

server <- "http://rest.ensembl.org/"
con <- "application/json"
ext_ping <- "/info/ping?"

x <- 0

while (x < 25) {
    ping <- GET(paste(server, ext_ping, sep = ""), accept(con))
    x = x + 1
    print( paste ("count: ", x, "status:", status_code(ping), "remaining:", headers(ping)['X-RateLimit-Remaining']))
}

## Exercises 7 – answers

This script queries the ping endpoint 25 times, printing the count, the HTTP Status Code, and the X-RateLimit-Remaining header each time.

1\. Increase the number of loops, do you start to get 429 errors?

2\. Can you add in a step to make it wait a few seconds every iteration? Or every 100 iterations?


In [ ]:
library(httr)

server <- "http://rest.ensembl.org/"
con <- "application/json"
ext_ping <- "/info/ping?"

x <- 0
y <- 0

while (x < 10000) {
    ping <- GET(paste(server, ext_ping, sep = ""), accept(con))
    x = x + 1
    y = y + 1
    print( paste ("count: ", x, "status:", status_code(ping), "remaining:", headers(ping)['X-RateLimit-Remaining']))
    
    if (y == 100) {
        Sys.sleep(10)
        y <-0
    }
}